In [1]:
import cv2
import numpy as np
import os
import dlib
import math
import tempfile

# Initialize face detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('model/shape_predictor_68_face_landmarks.dat')

otherImg = cv2.imread('filters/Carina_Nebula-1.png', cv2.IMREAD_UNCHANGED)
if otherImg is None: 
    print("other image not found.")

# Load accessory images
sunglasses = cv2.imread('filters/sunglasses.png', cv2.IMREAD_UNCHANGED)
if sunglasses is None:
    print("Sunglasses image not found.")
mustache = cv2.imread('filters/moustache.png', cv2.IMREAD_UNCHANGED)
if mustache is None:
    print("Mustache image not found.")
necktie = cv2.imread('filters/tie.png', cv2.IMREAD_UNCHANGED)
if necktie is None:
    print("Tie image not found.")
hat = cv2.imread('filters/straw_hat.png', cv2.IMREAD_UNCHANGED)
if hat is None:
    print("Hat image not found.")
shi = cv2.imread('filters/prof_shi.png', cv2.IMREAD_UNCHANGED)
if shi is None:
    print("Prof Shi image not found.")
shi2 = cv2.imread('filters/prof_shi2.png', cv2.IMREAD_UNCHANGED)
if shi2 is None:
    print("Prof Shi 2 image not found.")
hat2 = cv2.imread('filters/christmas_hat.png', cv2.IMREAD_UNCHANGED)
if hat2 is None:
    print("Hat 2 image not found.")

# Prof_Shi

In [2]:
def filter_shi(image):
    global detector, predictor, shi
    # Ensure face detection and buddy image are loaded
    if detector is None or predictor is None:
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # Adjust path

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if not faces:
        # No face detected, optionally place buddy in a default corner
        # or just return the original image
        return image

    # Take the first detected face
    face = faces[0]
    x_start, y_start, x_end, y_end = face.left(), face.top(), face.right(), face.bottom()

    face_height = y_end - y_start
    # Scale buddy image to have similar height as the face
    scale_factor = (face_height / shi.shape[0])* 2.5
    new_width = int(shi.shape[1] * scale_factor)
    new_height = int(shi.shape[0] * scale_factor)
    resized_buddy = cv2.resize(shi, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Decide placement: for example, place buddy to the right of the person's face
    # with some horizontal padding. You can also place to the left by changing sign.
    padding = 0
    x = x = x_start - new_width - padding
    y = y_start

    # Ensure coordinates are within the image
    x1, x2 = max(0, x), min(image.shape[1], x + new_width)
    y1, y2 = max(0, y), min(image.shape[0], y + new_height)

    # Adjust if buddy goes out of frame. If it doesn't fit, just clip it
    sx1, sx2 = x1 - x, x2 - x
    sy1, sy2 = y1 - y, y2 - y

    buddy_region = resized_buddy[sy1:sy2, sx1:sx2]

    if buddy_region.size == 0:
        return image  # If no space, skip

    # Blend the buddy image
    if buddy_region.shape[2] == 4:
        alpha_s = buddy_region[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        rgb_buddy = buddy_region[:, :, :3]
    else:
        alpha_s = np.ones((buddy_region.shape[0], buddy_region.shape[1]))
        alpha_l = 1.0 - alpha_s
        rgb_buddy = buddy_region

    roi = image[y1:y2, x1:x2]
    for c in range(0,3):
        roi[:,:,c] = (alpha_s*rgb_buddy[:,:,c] + alpha_l*roi[:,:,c])
    image[y1:y2, x1:x2] = roi

    return image

In [3]:
def filter_shi2(image):
    global detector, predictor, shi2
    # Ensure face detection and buddy image are loaded
    if detector is None or predictor is None:
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # Adjust path

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if not faces:
        # No face detected, optionally place buddy in a default corner
        # or just return the original image
        return image

    # Take the first detected face
    face = faces[0]
    x_start, y_start, x_end, y_end = face.left(), face.top(), face.right(), face.bottom()

    face_height = y_end - y_start
    # Scale buddy image to have similar height as the face
    scale_factor = (face_height / shi2.shape[0])* 2.5
    new_width = int(shi2.shape[1] * scale_factor)
    new_height = int(shi2.shape[0] * scale_factor)
    resized_buddy = cv2.resize(shi2, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Decide placement: for example, place buddy to the right of the person's face
    # with some horizontal padding. You can also place to the left by changing sign.
    padding = 20
    x = x = x_start - new_width - padding
    y = y_start

    # Ensure coordinates are within the image
    x1, x2 = max(0, x), min(image.shape[1], x + new_width)
    y1, y2 = max(0, y), min(image.shape[0], y + new_height)

    # Adjust if buddy goes out of frame. If it doesn't fit, just clip it
    sx1, sx2 = x1 - x, x2 - x
    sy1, sy2 = y1 - y, y2 - y

    buddy_region = resized_buddy[sy1:sy2, sx1:sx2]

    if buddy_region.size == 0:
        return image  # If no space, skip

    # Blend the buddy image
    if buddy_region.shape[2] == 4:
        alpha_s = buddy_region[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        rgb_buddy = buddy_region[:, :, :3]
    else:
        alpha_s = np.ones((buddy_region.shape[0], buddy_region.shape[1]))
        alpha_l = 1.0 - alpha_s
        rgb_buddy = buddy_region

    roi = image[y1:y2, x1:x2]
    for c in range(0,3):
        roi[:,:,c] = (alpha_s*rgb_buddy[:,:,c] + alpha_l*roi[:,:,c])
    image[y1:y2, x1:x2] = roi

    return image

# Sunglass

In [4]:
def filter_sunglass(image):
    global sunglasses, detector, predictor
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        # Extract eye landmarks
        left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
        right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])

        # Calculate eye centers
        left_eye_center = left_eye_points.mean(axis=0).astype("int")
        right_eye_center = right_eye_points.mean(axis=0).astype("int")

        # Calculate angle between the eye centers
        dy = right_eye_center[1] - left_eye_center[1]
        dx = right_eye_center[0] - left_eye_center[0]
        angle = -math.degrees(math.atan2(dy, dx))

        # Calculate the scaling factor based on the distance between the eyes
        eye_width = np.linalg.norm(right_eye_center - left_eye_center)
        scaling_factor = eye_width / sunglasses.shape[1] * 2.3  # Adjust scaling factor as needed

        # Resize the sunglasses image
        new_sunglasses_width = int(sunglasses.shape[1] * scaling_factor)
        new_sunglasses_height = int(sunglasses.shape[0] * scaling_factor)
        resized_sunglasses = cv2.resize(sunglasses, (new_sunglasses_width, new_sunglasses_height), interpolation=cv2.INTER_AREA)

        # Prepare for rotation without cropping
        # Calculate the center of the sunglasses image
        center = (new_sunglasses_width // 2, new_sunglasses_height // 2)

        # Get rotation matrix
        M = cv2.getRotationMatrix2D(center, angle, 1.0)

        # Calculate the sine and cosine of the rotation angle
        abs_cos = abs(M[0, 0])
        abs_sin = abs(M[0, 1])

        # Compute the new bounding dimensions of the image
        bound_w = int(new_sunglasses_width * abs_cos + new_sunglasses_height * abs_sin)
        bound_h = int(new_sunglasses_width * abs_sin + new_sunglasses_height * abs_cos)

        # Adjust the rotation matrix to consider the translation
        M[0, 2] += bound_w / 2 - center[0]
        M[1, 2] += bound_h / 2 - center[1]

        # Perform the actual rotation and prevent cropping
        rotated_sunglasses = cv2.warpAffine(resized_sunglasses, M, (bound_w, bound_h), flags=cv2.INTER_AREA, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

        # Calculate position to place sunglasses
        x = int((left_eye_center[0] + right_eye_center[0]) / 2 - bound_w / 2)
        y = int((left_eye_center[1] + right_eye_center[1]) / 2 - bound_h / 2)

        # Adjust position for better fit (optional)
        y += int(new_sunglasses_height * 0.1)  # Move sunglasses down slightly

        # Ensure coordinates are within the image
        x1, x2 = max(0, x), min(image.shape[1], x + bound_w)
        y1, y2 = max(0, y), min(image.shape[0], y + bound_h)

        # Calculate regions
        sunglasses_region = rotated_sunglasses[y1 - y:y2 - y, x1 - x:x2 - x]

        # Ensure the sunglasses region is not empty
        if sunglasses_region.size == 0:
            continue  # Skip if the region is empty

        # Split channels and compute the mask
        if sunglasses_region.shape[2] == 4:
            # If the image has an alpha channel
            alpha_s = sunglasses_region[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s
            # Extract the RGB channels
            sunglasses_rgb = sunglasses_region[:, :, :3]
        else:
            # If the image does not have an alpha channel
            alpha_s = np.ones((sunglasses_region.shape[0], sunglasses_region.shape[1]))
            alpha_l = 1.0 - alpha_s
            sunglasses_rgb = sunglasses_region

        # Extract the region of interest from the image
        roi = image[y1:y2, x1:x2]

        # Blend the sunglasses with the ROI
        for c in range(0, 3):
            roi[:, :, c] = (alpha_s * sunglasses_rgb[:, :, c] + alpha_l * roi[:, :, c])

        # Place the blended region back into the image
        image[y1:y2, x1:x2] = roi
    return image

# Mustache

In [5]:
def filter_mustache(image):
  global mustache, detector, predictor

  # Read the image
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Detect faces
  faces = detector(gray)

  for face in faces:
      landmarks = predictor(gray, face)

      # Extract eye landmarks
      left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 51)])
      right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(52, 55)])

      # Calculate eye centers
      left_eye_center = left_eye_points.mean(axis=0).astype("int")
      right_eye_center = right_eye_points.mean(axis=0).astype("int")

      # Calculate angle between the eye centers
      dy = right_eye_center[1] - left_eye_center[1]
      dx = right_eye_center[0] - left_eye_center[0]
      angle = -math.degrees(math.atan2(dy, dx))

      # Calculate the scaling factor based on the distance between the eyes
      eye_width = np.linalg.norm(right_eye_center - left_eye_center)
      scaling_factor = eye_width / mustache.shape[1] * 2.0  # Adjust scaling factor as needed

      # Resize the sunglasses image
      new_sunglasses_width = int(mustache.shape[1] * scaling_factor)
      new_sunglasses_height = int(mustache.shape[0] * scaling_factor)
      resized_sunglasses = cv2.resize(mustache, (new_sunglasses_width, new_sunglasses_height), interpolation=cv2.INTER_AREA)

      # Prepare for rotation without cropping
      # Calculate the center of the sunglasses image
      center = (new_sunglasses_width // 2, new_sunglasses_height // 2)

      # Get rotation matrix
      M = cv2.getRotationMatrix2D(center, angle, 1.0)

      # Calculate the sine and cosine of the rotation angle
      abs_cos = abs(M[0, 0])
      abs_sin = abs(M[0, 1])

      # Compute the new bounding dimensions of the image
      bound_w = int(new_sunglasses_width * abs_cos + new_sunglasses_height * abs_sin)
      bound_h = int(new_sunglasses_width * abs_sin + new_sunglasses_height * abs_cos)

      # Adjust the rotation matrix to consider the translation
      M[0, 2] += bound_w / 2 - center[0]
      M[1, 2] += bound_h / 2 - center[1]

      # Perform the actual rotation and prevent cropping
      rotated_sunglasses = cv2.warpAffine(resized_sunglasses, M, (bound_w, bound_h), flags=cv2.INTER_AREA, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

      # Calculate position to place sunglasses
      x = int((left_eye_center[0] + right_eye_center[0]) / 2 - bound_w / 2)
      y = int((left_eye_center[1] + right_eye_center[1]) / 2 - bound_h / 2)

      # Ensure coordinates are within the image
      x1, x2 = max(0, x), min(image.shape[1], x + bound_w)
      y1, y2 = max(0, y), min(image.shape[0], y + bound_h)

      # Calculate regions
      sunglasses_region = rotated_sunglasses[y1 - y:y2 - y, x1 - x:x2 - x]

      # Ensure the sunglasses region is not empty
      if sunglasses_region.size == 0:
          continue  # Skip if the region is empty

      # Split channels and compute the mask
      if sunglasses_region.shape[2] == 4:
          # If the image has an alpha channel
          alpha_s = sunglasses_region[:, :, 3] / 255.0
          alpha_l = 1.0 - alpha_s
          # Extract the RGB channels
          sunglasses_rgb = sunglasses_region[:, :, :3]
      else:
          # If the image does not have an alpha channel
          alpha_s = np.ones((sunglasses_region.shape[0], sunglasses_region.shape[1]))
          alpha_l = 1.0 - alpha_s
          sunglasses_rgb = sunglasses_region

      # Extract the region of interest from the image
      roi = image[y1:y2, x1:x2]

      # Blend the sunglasses with the ROI
      for c in range(0, 3):
          roi[:, :, c] = (alpha_s * sunglasses_rgb[:, :, c] + alpha_l * roi[:, :, c])

      # Place the blended region back into the image
      image[y1:y2, x1:x2] = roi

  return image

# Necktie

In [6]:
def filter_necktie(image):
  # FOR THE NECKTIE
    global necktie, detector, predictor

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        # Extract eye landmarks
        left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(7, 8)])
        right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(8, 9)])

        # Calculate eye centers
        left_eye_center = left_eye_points.mean(axis=0).astype("int")
        right_eye_center = right_eye_points.mean(axis=0).astype("int")

        # Calculate angle between the eye centers
        dy = right_eye_center[1] - left_eye_center[1]
        dx = right_eye_center[0] - left_eye_center[0]
        angle = -math.degrees(math.atan2(dy, dx))

        # Calculate the scaling factor based on the distance between the eyes
        eye_width = np.linalg.norm(right_eye_center - left_eye_center)
        scaling_factor = eye_width / necktie.shape[1] * 10.0  # Adjust scaling factor for tie!!

        # Resize the sunglasses image
        new_sunglasses_width = int(necktie.shape[1] * scaling_factor)
        new_sunglasses_height = int(necktie.shape[0] * scaling_factor)
        resized_sunglasses = cv2.resize(necktie, (new_sunglasses_width, new_sunglasses_height), interpolation=cv2.INTER_AREA)

        # Prepare for rotation without cropping
        # Calculate the center of the sunglasses image
        center = (new_sunglasses_width // 2, 0)

        # Get rotation matrix
        M = cv2.getRotationMatrix2D(center, angle, 1.0)

        # Calculate the sine and cosine of the rotation angle
        abs_cos = abs(M[0, 0])
        abs_sin = abs(M[0, 1])

        # Compute the new bounding dimensions of the image
        bound_w = int(new_sunglasses_width * abs_cos + new_sunglasses_height * abs_sin)
        bound_h = int(new_sunglasses_width * abs_sin + new_sunglasses_height * abs_cos)

        # Adjust the rotation matrix to consider the translation
        M[0, 2] += bound_w / 2 - center[0]
        M[1, 2] += bound_h / 2 - center[1]

        # Perform the actual rotation and prevent cropping
        rotated_sunglasses = cv2.warpAffine(resized_sunglasses, M, (bound_w, bound_h), flags=cv2.INTER_AREA, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

        # Calculate position to place sunglasses
        x = int((left_eye_center[0] + right_eye_center[0]) / 2 - bound_w / 2)
        y = int((left_eye_center[1] + right_eye_center[1]) / 2 - bound_h / 2)

        # Adjust position for better fit (optional)
        y += int(new_sunglasses_height * 0.1)  # Move sunglasses down slightly

        # Ensure coordinates are within the image
        x1, x2 = max(0, x), min(image.shape[1], x + bound_w)
        y1, y2 = max(0, y), min(image.shape[0], y + bound_h)

        # Calculate regions
        sunglasses_region = rotated_sunglasses[y1 - y:y2 - y, x1 - x:x2 - x]

        # Ensure the sunglasses region is not empty
        if sunglasses_region.size == 0:
            continue  # Skip if the region is empty

        # Split channels and compute the mask
        if sunglasses_region.shape[2] == 4:
            # If the image has an alpha channel
            alpha_s = sunglasses_region[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s
            # Extract the RGB channels
            sunglasses_rgb = sunglasses_region[:, :, :3]
        else:
            # If the image does not have an alpha channel
            alpha_s = np.ones((sunglasses_region.shape[0], sunglasses_region.shape[1]))
            alpha_l = 1.0 - alpha_s
            sunglasses_rgb = sunglasses_region

        # Extract the region of interest from the image
        roi = image[y1:y2, x1:x2]

        # Blend the sunglasses with the ROI
        for c in range(0, 3):
            roi[:, :, c] = (alpha_s * sunglasses_rgb[:, :, c] + alpha_l * roi[:, :, c])

        # Place the blended region back into the image
        image[y1:y2, x1:x2] = roi
    return image

# Straw Hat

In [7]:
def filter_hat(image):
    """
    Overlays a hat on detected faces in the image.

    Parameters:
        image (np.ndarray): The input image in BGR format.

    Returns:
        np.ndarray: The image with hats overlayed.
    """
    global hat, detector, predictor
    # Make a copy to avoid modifying the original image
    output_image = image.copy()

    # Convert image to grayscale for face detection
    gray = cv2.cvtColor(output_image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)

    for face in faces:
        try:
            landmarks = predictor(gray, face)

            # Extract eye landmarks
            left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
            right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])

            # Calculate eye centers
            left_eye_center = left_eye_points.mean(axis=0).astype("int")
            right_eye_center = right_eye_points.mean(axis=0).astype("int")

            # Calculate angle between the eye centers
            dy = right_eye_center[1] - left_eye_center[1]
            dx = right_eye_center[0] - left_eye_center[0]
            angle_rad = np.arctan2(dy, dx)  # Angle in radians
            angle_deg = np.degrees(angle_rad)  # Angle in degrees

            # Calculate the scaling factor based on the face width
            # Store original hat dimensions for scaling
            hat_original_width = hat.shape[1]
            hat_original_height = hat.shape[0]
            face_width = face.right() - face.left()
            scaling_factor = face_width / hat_original_width * 1.5  # Adjust multiplier to control hat size

            # Resize the hat image
            new_hat_width = int(hat_original_width * scaling_factor)
            new_hat_height = int(hat_original_height * scaling_factor)
            resized_hat = cv2.resize(hat, (new_hat_width, new_hat_height), interpolation=cv2.INTER_AREA)

            # Rotate the hat image
            center = (new_hat_width // 2, new_hat_height // 2)
            M = cv2.getRotationMatrix2D(center, angle_deg, 1.0)

            # Compute the sine and cosine of the rotation angle
            cos = np.abs(M[0, 0])
            sin = np.abs(M[0, 1])

            # Compute the new bounding dimensions of the image
            nW = int((new_hat_height * sin) + (new_hat_width * cos))
            nH = int((new_hat_height * cos) + (new_hat_width * sin))

            # Adjust the rotation matrix to take into account translation
            M[0, 2] += (nW / 2) - center[0]
            M[1, 2] += (nH / 2) - center[1]

            # Perform the actual rotation and get the rotated hat
            rotated_hat = cv2.warpAffine(resized_hat, M, (nW, nH), flags=cv2.INTER_AREA,
                                         borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

            # Compute the midpoint between the eyes
            eyes_center = ((left_eye_center[0] + right_eye_center[0]) // 2,
                           (left_eye_center[1] + right_eye_center[1]) // 2)

            # Compute the offset magnitude (adjust as needed)
            offset_magnitude = face_width * 0.6  # Adjust this multiplier to control how far up the hat is placed

            # Calculate the offset in the rotated coordinate system
            dx_offset = -offset_magnitude * np.sin(angle_rad)
            dy_offset = -offset_magnitude * np.cos(angle_rad)

            # Compute the final position where the hat should be placed
            x = int(eyes_center[0] - (nW / 2) + dx_offset)
            y = int(eyes_center[1] - (nH / 2) + dy_offset)

            # Ensure the coordinates are within the image bounds
            x1 = max(0, x)
            y1 = max(0, y)
            x2 = min(output_image.shape[1], x + nW)
            y2 = min(output_image.shape[0], y + nH)

            # Compute the corresponding region in the rotated hat image
            hat_x1 = max(0, -x)
            hat_y1 = max(0, -y)
            hat_x2 = hat_x1 + (x2 - x1)
            hat_y2 = hat_y1 + (y2 - y1)

            # Crop the rotated hat and the ROI (Region of Interest) to be blended
            hat_cropped = rotated_hat[hat_y1:hat_y2, hat_x1:hat_x2]
            roi = output_image[y1:y2, x1:x2]

            # Check if the overlay and roi are valid
            if hat_cropped.shape[0] == 0 or hat_cropped.shape[1] == 0 or roi.shape[0] == 0 or roi.shape[1] == 0:
                continue

            # Separate the color and alpha channels
            if hat_cropped.shape[2] == 4:
                alpha_s = hat_cropped[:, :, 3] / 255.0
                alpha_l = 1.0 - alpha_s
                overlay_bgr = hat_cropped[:, :, :3]
            else:
                # If no alpha channel, assume full opacity
                alpha_s = np.ones((hat_cropped.shape[0], hat_cropped.shape[1]))
                alpha_l = 1.0 - alpha_s
                overlay_bgr = hat_cropped

            # Reshape alpha channels for broadcasting
            alpha_s = alpha_s[:, :, np.newaxis]
            alpha_l = alpha_l[:, :, np.newaxis]

            # Blend the overlay with the ROI using vectorized operations
            blended = (alpha_s * overlay_bgr + alpha_l * roi).astype(np.uint8)

            # Place the blended region back into the main image
            output_image[y1:y2, x1:x2] = blended
        except:
            return image
    return output_image

In [8]:
def filter_hat2(image):
    """
    Overlays a hat on detected faces in the image.

    Parameters:
        image (np.ndarray): The input image in BGR format.

    Returns:
        np.ndarray: The image with hats overlayed.
    """
    global hat2, detector, predictor
    # Make a copy to avoid modifying the original image
    output_image = image.copy()

    # Convert image to grayscale for face detection
    gray = cv2.cvtColor(output_image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)

    for face in faces:
        try:
            landmarks = predictor(gray, face)

            # Extract eye landmarks
            left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
            right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])

            # Calculate eye centers
            left_eye_center = left_eye_points.mean(axis=0).astype("int")
            right_eye_center = right_eye_points.mean(axis=0).astype("int")

            # Calculate angle between the eye centers
            dy = right_eye_center[1] - left_eye_center[1]
            dx = right_eye_center[0] - left_eye_center[0]
            angle_rad = np.arctan2(dy, dx)  # Angle in radians
            angle_deg = np.degrees(angle_rad)  # Angle in degrees

            # Calculate the scaling factor based on the face width
            # Store original hat dimensions for scaling
            hat_original_width = hat2.shape[1]
            hat_original_height = hat2.shape[0]
            face_width = face.right() - face.left()
            scaling_factor = face_width / hat_original_width * 1.5  # Adjust multiplier to control hat size

            # Resize the hat image
            new_hat_width = int(hat_original_width * scaling_factor)
            new_hat_height = int(hat_original_height * scaling_factor)
            resized_hat = cv2.resize(hat2, (new_hat_width, new_hat_height), interpolation=cv2.INTER_AREA)

            # Rotate the hat image
            center = (new_hat_width // 2, new_hat_height // 2)
            M = cv2.getRotationMatrix2D(center, angle_deg, 1.0)

            # Compute the sine and cosine of the rotation angle
            cos = np.abs(M[0, 0])
            sin = np.abs(M[0, 1])

            # Compute the new bounding dimensions of the image
            nW = int((new_hat_height * sin) + (new_hat_width * cos))
            nH = int((new_hat_height * cos) + (new_hat_width * sin))

            # Adjust the rotation matrix to take into account translation
            M[0, 2] += (nW / 2) - center[0]
            M[1, 2] += (nH / 2) - center[1]

            # Perform the actual rotation and get the rotated hat
            rotated_hat = cv2.warpAffine(resized_hat, M, (nW, nH), flags=cv2.INTER_AREA,
                                         borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

            # Compute the midpoint between the eyes
            eyes_center = ((left_eye_center[0] + right_eye_center[0]) // 2,
                           (left_eye_center[1] + right_eye_center[1]) // 2)

            # Compute the offset magnitude (adjust as needed)
            offset_magnitude = face_width * 0.6  # Adjust this multiplier to control how far up the hat is placed

            # Calculate the offset in the rotated coordinate system
            dx_offset = -offset_magnitude * np.sin(angle_rad)
            dy_offset = -offset_magnitude * np.cos(angle_rad)

            # Compute the final position where the hat should be placed
            x = int(eyes_center[0] - (nW / 2) + dx_offset)
            y = int(eyes_center[1] - (nH / 2) + dy_offset)

            # Ensure the coordinates are within the image bounds
            x1 = max(0, x)
            y1 = max(0, y)
            x2 = min(output_image.shape[1], x + nW)
            y2 = min(output_image.shape[0], y + nH)

            # Compute the corresponding region in the rotated hat image
            hat_x1 = max(0, -x)
            hat_y1 = max(0, -y)
            hat_x2 = hat_x1 + (x2 - x1)
            hat_y2 = hat_y1 + (y2 - y1)

            # Crop the rotated hat and the ROI (Region of Interest) to be blended
            hat_cropped = rotated_hat[hat_y1:hat_y2, hat_x1:hat_x2]
            roi = output_image[y1:y2, x1:x2]

            # Check if the overlay and roi are valid
            if hat_cropped.shape[0] == 0 or hat_cropped.shape[1] == 0 or roi.shape[0] == 0 or roi.shape[1] == 0:
                continue

            # Separate the color and alpha channels
            if hat_cropped.shape[2] == 4:
                alpha_s = hat_cropped[:, :, 3] / 255.0
                alpha_l = 1.0 - alpha_s
                overlay_bgr = hat_cropped[:, :, :3]
            else:
                # If no alpha channel, assume full opacity
                alpha_s = np.ones((hat_cropped.shape[0], hat_cropped.shape[1]))
                alpha_l = 1.0 - alpha_s
                overlay_bgr = hat_cropped

            # Reshape alpha channels for broadcasting
            alpha_s = alpha_s[:, :, np.newaxis]
            alpha_l = alpha_l[:, :, np.newaxis]

            # Blend the overlay with the ROI using vectorized operations
            blended = (alpha_s * overlay_bgr + alpha_l * roi).astype(np.uint8)

            # Place the blended region back into the main image
            output_image[y1:y2, x1:x2] = blended
        except:
            return image
    return output_image

# Shades

In [9]:
from scipy import signal
def filter_sepia(image):
    sepia_filter = np.array([[0.272, 0.534, 0.131],
                             [0.349, 0.686, 0.168],
                             [0.393, 0.769, 0.189]])
    sepia_image = cv2.transform(image, sepia_filter)
    sepia_image = np.clip(sepia_image, 0, 255).astype(np.uint8)
    return sepia_image

def filter_negative(image):
    return cv2.bitwise_not(image)


In [10]:
def apply_gaussian_filter(image):
    kernel = np.array([[0.0030, 0.0133, 0.0219, 0.0133, 0.0030],
                       [0.0133, 0.0596, 0.0983, 0.0596, 0.0133],
                       [0.0219, 0.0983, 0.1621, 0.0983, 0.0219],
                       [0.0133, 0.0596, 0.0983, 0.0596, 0.0133],
                       [0.0030, 0.0133, 0.0219, 0.0133, 0.0030]], dtype="float")
    out_img = image.copy()

    for c in range(3):
        channel = out_img[:, :, c]
        gauss = signal.convolve2d(channel, kernel, mode='same', boundary='symm')
        gauss = np.clip(gauss, 0, 255).astype("uint8")
        out_img[:, :, c] = gauss

    return out_img

def apply_sobel_filter(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gx = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]], dtype="int")
    gy = np.array([[1, 2, 1],
                   [0, 0, 0],
                   [-1, -2, -1]], dtype="int")
    sobelx = signal.convolve2d(gray, gx, mode='same', boundary='symm')
    sobely = signal.convolve2d(gray, gy, mode='same', boundary='symm')
    sobel = np.sqrt(sobelx**2 + sobely**2)
    sobel = np.clip(sobel, 0, 255).astype("uint8")
    # Return as a BGR image
    out_img = cv2.cvtColor(sobel, cv2.COLOR_GRAY2BGR)
    return out_img

def apply_filmgrain_filter(image):
    out_img = image.copy().astype(np.float32)
    h, w, _ = out_img.shape
    rand_nums = np.random.randint(0, 100, size=(h, w))

    # Apply varying brightness changes
    # <20: *0.8, <40: *0.6, <60: *0.5, <80: *1.2, <100: *1.4
    mask_20 = rand_nums < 20
    mask_40 = (rand_nums >= 20) & (rand_nums < 40)
    mask_60 = (rand_nums >= 40) & (rand_nums < 60)
    mask_80 = (rand_nums >= 60) & (rand_nums < 80)
    mask_100 = (rand_nums >= 80)

    out_img[mask_20] *= 0.8
    out_img[mask_40] *= 0.6
    out_img[mask_60] *= 0.5
    out_img[mask_80] *= 1.2
    out_img[mask_100] *= 1.4

    out_img = np.clip(out_img, 0, 255).astype(np.uint8)
    return out_img

def apply_pixelate_filter(image):
    out_img = image.copy()
    pixel_size = 5
    kernel = np.ones((3, 3)) / 9

    # Downsample
    downsample = out_img[::pixel_size, ::pixel_size].astype(np.float32)
    for c in range(3):
        downsample[:, :, c] = signal.convolve2d(downsample[:, :, c], kernel, mode='same', boundary='symm')

    # Upsample
    pixelated_image = np.repeat(np.repeat(downsample, pixel_size, axis=0), pixel_size, axis=1)
    pixelated_image = np.clip(pixelated_image, 0, 255).astype(np.uint8)
    return pixelated_image

def apply_dither_filter(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    for y in range(1, gray.shape[1]-1):
        for x in range(1, gray.shape[0]-1):
            oldpixel = gray[x, y]
            if oldpixel > 128:
                newpixel = 255
            else:
                newpixel = 0
            gray[x, y] = newpixel
            quant_error = oldpixel - newpixel
            
            gray[x + 1, y] += quant_error * (7 / 16)
            gray[x - 1, y + 1] += quant_error * (3 / 16)
            gray[x, y + 1] += quant_error * (5 / 16)
            gray[x + 1, y + 1] += quant_error * (1 / 16)
        
    image = gray 
    return image

def apply_bitmap_filter(image):
    # Using the final "dither" from code
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32)
    height, width = gray.shape

    for y in range(height - 1):
        old_row = gray[y, :]
        new_row = np.where(old_row > 96, 255, 0)
        gray[y, :] = new_row
        quant_error = old_row - new_row

        if y + 1 < height:
            # bottom-left
            if width > 1:
                gray[y + 1, :-1] += quant_error[1:] * (3 / 16)
            gray[y + 1, :] += quant_error * (5 / 16)
            if width > 1:
                gray[y + 1, 1:] += quant_error[:-1] * (1 / 16)

        if width > 1:
            # right neighbor
            gray[y, 1:] += quant_error[:-1] * (7 / 16)

    dithered = np.clip(gray, 0, 255).astype(np.uint8)
    out_img = cv2.cvtColor(dithered, cv2.COLOR_GRAY2BGR)
    return out_img

def apply_kuwahara_filter(image, window_size=9):
    """
    Apply the Kuwahara filter to the input image using integral images for optimization.
    
    Parameters:
    - image: Input image in BGR format as a NumPy array.
    - window_size: Size of the window (must be odd, e.g., 5, 7). Larger sizes capture more context but are slower.
    
    Returns:
    - out_img: Filtered image in BGR format as a NumPy array.
    """
    if window_size % 2 == 0:
        raise ValueError("Window size must be odd.")
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32)
    
    imageFloat = gray.astype(np.float64)

    winsize = 9

    #Build subwindows
    tmpAvgKerRow = np.hstack((np.ones((1,int((winsize-1)/2+1))),np.zeros((1,int((winsize-1)/2)))))
    tmpPadder = np.zeros((1,winsize))
    tmpavgker = np.tile(tmpAvgKerRow, (int((winsize-1)/2+1),1))
    tmpavgker = np.vstack((tmpavgker, np.tile(tmpPadder, (int((winsize-1)/2),1))))
    tmpavgker = tmpavgker/np.sum(tmpavgker)

    # tmpavgker is a 'north-west' subwindow (marked as 'a' above)
    # we build a vector of convolution kernels for computing average and
    # variance
    avgker = np.empty((4,winsize,winsize)) # make an empty vector of arrays
    avgker[0] = tmpavgker			# North-west (a)
    avgker[1] = np.fliplr(tmpavgker)	# North-east (b)
    avgker[2] = np.flipud(tmpavgker)	# South-west (c)
    avgker[3] = np.fliplr(avgker[2])	# South-east (d)
    
    # Create a pixel-by-pixel square of the image
    squaredImg = imageFloat**2
    
    # preallocate these arrays to make it apparently %15 faster
    avgs = np.zeros([4, imageFloat.shape[0],imageFloat.shape[1]])
    stddevs = avgs.copy()

    # Calculation of averages and variances on subwindows
    for k in range(4):
        avgs[k] = signal.convolve2d(imageFloat, avgker[k],mode='same') 	    # mean on subwindow
        stddevs[k] = signal.convolve2d(squaredImg, avgker[k],mode='same')  # mean of squares on subwindow
        stddevs[k] = stddevs[k]-avgs[k]**2 			    # variance on subwindow
    
    # Choice of index with minimum variance
    indices = np.argmin(stddevs,0) # returns index of subwindow with smallest variance

    # Building the filtered image (with nested for loops)
    filtered = np.zeros(image.shape)
    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            filtered[row,col] = avgs[indices[row,col], row,col]

    #filtered=filtered.astype(np.uint8)
    image = filtered.astype(np.uint8)
    
    return image

def apply_blendframes_filter(image, alpha=0.8):
    global otherImg
    # otherImg must be same size or we resize it
    if otherImg.shape[:2] != image.shape[:2]:
        otherImg = cv2.resize(otherImg, (image.shape[1], image.shape[0]))
    out_img = image.astype(np.float32)
    out_img = out_img * alpha + otherImg.astype(np.float32) * (1 - alpha)
    out_img = np.clip(out_img, 0, 255).astype(np.uint8)
    return out_img

def apply_brokenvcr_filter(image):
    # Adapted from provided code
    PI = 3.14159
    test = (0.25 * 0.5) % 1.0
    warped_image = image.astype(np.float32)
    h, w, _ = warped_image.shape
    y_coords = np.linspace(0, 1, h)
    y_diff = np.abs(y_coords - test)
    mask = y_diff < 0.5

    for y in range(h):
        if mask[y]:
            warped_image[y, :, 1] += 0.9 * np.tan((y / h) * 150.0)
    warped_image = np.clip(warped_image, 0, 255)

    red_copy = np.copy(warped_image[:, :, 0])
    if h > 10 and w > 10:
        red_copy[10:, 10:] = warped_image[:-10, :-10, 0]
        red_copy[:10, :] = 0
        red_copy[:, :10] = 0

    green_copy = np.copy(warped_image[:, :, 1])
    if h > 5 and w > 5:
        green_copy[5:, 5:] = warped_image[:-5, :-5, 1]
        green_copy[:5, :] = 0
        green_copy[:, :5] = 0

    shifted_image = warped_image.copy()
    shifted_image[:, :, 0] = red_copy
    shifted_image[:, :, 1] = green_copy

    out_img = np.clip(shifted_image, 0, 255).astype(np.uint8)
    return out_img


# UI

In [11]:
shade_filters = {
    "None": lambda img: img,
    "Sepia": filter_sepia,
    "Negative": filter_negative,
    "Gaussian": apply_gaussian_filter,
    "Edges": apply_sobel_filter,
    "Filmgrain": apply_filmgrain_filter,
    "Pixelate": apply_pixelate_filter,
    "Dither": apply_dither_filter,
    "Bit Map": apply_bitmap_filter,
    "Kuwahara": apply_kuwahara_filter,
    "BlendFrames": apply_blendframes_filter,  # Remember to pass otherImg as arg in UI code
    "Brokenvcr": apply_brokenvcr_filter
}

filter_functions = {
    "None": lambda img: img,
    "Sunglass": filter_sunglass,
    "Mustache": filter_mustache,
    "Necktie": filter_necktie,
    "Straw Hat": filter_hat,
    "Prof_Shi": filter_shi,
    "Prof_Shi2": filter_shi2,
    "Christmas Hat": filter_hat2,
}

In [12]:
import sys
import os
import tempfile
import cv2
import numpy as np
import shutil

from PyQt5.QtWidgets import (QApplication, QWidget, QVBoxLayout, QHBoxLayout,
                             QLabel, QPushButton, QComboBox, QFileDialog, QMessageBox)
from PyQt5.QtCore import QTimer, Qt
from PyQt5.QtGui import QImage, QPixmap

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Overlay + Shade Filters - PyQt")
        self.resize(800, 600)

        self.input_mode = "Image"
        self.current_image = None
        self.current_video_path = None
        self.cap = None
        self.timer = QTimer()

        self.processed_video_output_path = None
        self.last_webcam_frame = None
        self.flip_webcam = False

        # Combos for overlay and shade filters
        self.overlay_combo = QComboBox()
        self.overlay_combo.addItems(list(filter_functions.keys()))
        
        self.shade_combo = QComboBox()
        self.shade_combo.addItems(list(shade_filters.keys()))

        self.mode_combo = QComboBox()
        self.mode_combo.addItems(["Image", "Video", "Webcam"])
        self.mode_combo.currentTextChanged.connect(self.on_mode_changed)

        self.load_button = QPushButton("Load")
        self.load_button.clicked.connect(self.on_load_clicked)

        self.apply_button = QPushButton("Apply Filters")
        self.apply_button.clicked.connect(self.on_apply_filters)
        
        self.save_button = QPushButton("Save")
        self.save_button.setEnabled(False)
        self.save_button.clicked.connect(self.on_save_clicked)

        self.take_snapshot_button = QPushButton("Take Snapshot")
        self.take_snapshot_button.setEnabled(False)
        self.take_snapshot_button.clicked.connect(self.on_take_snapshot_clicked)

        self.flip_button = QPushButton("Flip Horizontal")
        self.flip_button.setEnabled(False)
        self.flip_button.clicked.connect(self.on_flip_clicked)

        self.image_label = QLabel("No input loaded.")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setStyleSheet("background-color: #aaa;")

        top_layout = QHBoxLayout()
        top_layout.addWidget(QLabel("Input Type:"))
        top_layout.addWidget(self.mode_combo)
        top_layout.addSpacing(20)

        top_layout.addWidget(QLabel("Overlay Filter:"))
        top_layout.addWidget(self.overlay_combo)
        top_layout.addSpacing(20)

        top_layout.addWidget(QLabel("Shade Filter:"))
        top_layout.addWidget(self.shade_combo)
        top_layout.addSpacing(20)

        top_layout.addWidget(self.load_button)
        top_layout.addWidget(self.apply_button)
        top_layout.addWidget(self.take_snapshot_button)
        top_layout.addWidget(self.flip_button)
        top_layout.addWidget(self.save_button)

        main_layout = QVBoxLayout()
        main_layout.addLayout(top_layout)
        main_layout.addWidget(self.image_label, stretch=1)

        self.setLayout(main_layout)
        self.update_ui_for_mode()

    def on_mode_changed(self, text):
        self.input_mode = text
        self.update_ui_for_mode()

    def update_ui_for_mode(self):
        self.save_button.setEnabled(False)
        self.take_snapshot_button.setEnabled(False)
        self.flip_button.setEnabled(False)
        if self.input_mode == "Image":
            self.load_button.setText("Load Image")
            self.stop_webcam()
            self.image_label.setText("No image loaded.")
            self.current_image = None
            self.current_video_path = None
        elif self.input_mode == "Video":
            self.load_button.setText("Load Video")
            self.stop_webcam()
            self.image_label.setText("No video loaded.")
            self.current_image = None
            self.current_video_path = None
        elif self.input_mode == "Webcam":
            self.load_button.setText("Start Webcam")
            self.current_image = None
            self.current_video_path = None
            self.start_webcam()
            self.take_snapshot_button.setEnabled(True)
            self.flip_button.setEnabled(True)

    def on_load_clicked(self):
        self.save_button.setEnabled(False)
        if self.input_mode == "Image":
            path, _ = QFileDialog.getOpenFileName(self, "Select Image", "", "Images (*.png *.jpg *.jpeg *.bmp)")
            if path:
                img = cv2.imread(path)
                if img is not None:
                    self.current_image = img
                    self.show_image(img)
                else:
                    QMessageBox.warning(self, "Error", "Failed to load image.")
        elif self.input_mode == "Video":
            path, _ = QFileDialog.getOpenFileName(self, "Select Video", "", "Video Files (*.mp4 *.avi *.mov)")
            if path:
                self.current_video_path = path
                cap = cv2.VideoCapture(path)
                ret, frame = cap.read()
                cap.release()
                if ret:
                    self.show_image(frame)
                else:
                    QMessageBox.warning(self, "Error", "Failed to load video.")
        elif self.input_mode == "Webcam":
            # Toggle webcam on/off
            if self.cap is None:
                self.start_webcam()
            else:
                self.stop_webcam()

    def start_webcam(self):
        if self.cap is not None:
            return
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            QMessageBox.warning(self, "Error", "Failed to access webcam.")
            self.cap = None
            return
        self.timer.timeout.connect(self.update_webcam_frame)
        self.timer.start(30)
        self.load_button.setText("Stop Webcam")
        self.take_snapshot_button.setEnabled(True)
        self.flip_button.setEnabled(True)

    def stop_webcam(self):
        if self.cap is not None:
            self.timer.stop()
            self.cap.release()
            self.cap = None
            self.load_button.setText("Start Webcam")
            self.image_label.setText("Webcam stopped.")
            self.take_snapshot_button.setEnabled(False)
            self.flip_button.setEnabled(False)

    def update_webcam_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return
        if self.flip_webcam:
            frame = cv2.flip(frame, 1)
        frame = self.apply_filters_to_frame(frame)
        self.show_image(frame)
        self.last_webcam_frame = frame

    def apply_filters_to_frame(self, frame):
        # Apply overlay filter first
        overlay_name = self.overlay_combo.currentText()
        overlay_func = filter_functions.get(overlay_name, lambda x: x)
        frame = overlay_func(frame)

        # Then apply shade filter
        shade_name = self.shade_combo.currentText()
        shade_func = shade_filters.get(shade_name, lambda x: x)
        frame = shade_func(frame)

        return frame

    def show_image(self, image):
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb.shape
        bytes_per_line = ch * w
        qimg = QImage(rgb.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pix = QPixmap.fromImage(qimg)
        self.image_label.setPixmap(pix.scaled(self.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))

    def on_apply_filters(self):
        if self.input_mode == "Image":
            if self.current_image is None:
                QMessageBox.warning(self, "No Image", "Load an image first.")
                return
            processed = self.apply_filters_to_frame(self.current_image)
            self.show_image(processed)
            self.current_image = processed
            self.save_button.setEnabled(True)
        elif self.input_mode == "Video":
            if self.current_video_path is None:
                QMessageBox.warning(self, "No Video", "Load a video first.")
                return
            output_path = self.process_video_file(self.current_video_path)
            if output_path:
                # Show first frame of processed video
                cap = cv2.VideoCapture(output_path)
                ret, frame = cap.read()
                cap.release()
                if ret:
                    self.show_image(frame)
                QMessageBox.information(self, "Done", f"Video processed at:\n{output_path}")
                self.processed_video_output_path = output_path
                self.save_button.setEnabled(True)
            else:
                QMessageBox.warning(self, "Error", "Video processing failed.")
        elif self.input_mode == "Webcam":
            QMessageBox.information(self, "Info", "Filter applied in real-time to webcam feed.\nUse 'Take Snapshot' to capture a frame.")

    def process_video_file(self, video_path):
        overlay_name = self.overlay_combo.currentText()
        overlay_func = filter_functions.get(overlay_name, lambda x: x)

        shade_name = self.shade_combo.currentText()
        shade_func = shade_filters.get(shade_name, lambda x: x)

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return None

        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        temp_dir = tempfile.gettempdir()
        output_path = os.path.join(temp_dir, 'processed_video_output.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = overlay_func(frame)
            frame = shade_func(frame)
            if len(frame.shape) == 2:
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
            out.write(frame)

        cap.release()
        out.release()
        return output_path

    def on_save_clicked(self):
        if self.input_mode == "Image" and self.current_image is not None:
            path, _ = QFileDialog.getSaveFileName(self, "Save Image", "", "Images (*.png *.jpg *.jpeg *.bmp)")
            if path:
                cv2.imwrite(path, self.current_image)
                QMessageBox.information(self, "Saved", f"Image saved at:\n{path}")
        elif self.input_mode == "Video" and self.processed_video_output_path is not None:
            path, _ = QFileDialog.getSaveFileName(self, "Save Video", "", "Video Files (*.mp4 *.avi)")
            if path:
                try:
                    shutil.copyfile(self.processed_video_output_path, path)
                    QMessageBox.information(self, "Saved", f"Video saved at:\n{path}")
                except Exception as e:
                    QMessageBox.warning(self, "Error", f"Failed to save video.\n{e}")
        elif self.input_mode == "Webcam" and self.current_image is not None:
            path, _ = QFileDialog.getSaveFileName(self, "Save Snapshot", "", "Images (*.png *.jpg *.jpeg *.bmp)")
            if path:
                cv2.imwrite(path, self.current_image)
                QMessageBox.information(self, "Saved", f"Snapshot saved at:\n{path}")
        else:
            QMessageBox.information(self, "Info", "Nothing to save at this moment.")

    def on_take_snapshot_clicked(self):
        # If we have last_webcam_frame stored, treat it as current_image.
        if self.input_mode == "Webcam" and self.last_webcam_frame is not None:
            self.current_image = self.last_webcam_frame.copy()
            self.save_button.setEnabled(True)
            QMessageBox.information(self, "Snapshot", "Snapshot taken and ready to save!")
        else:
            QMessageBox.information(self, "Info", "No webcam frame available to snapshot.")

    def on_flip_clicked(self):
        self.flip_webcam = not self.flip_webcam
        if self.flip_webcam:
            self.flip_button.setText("Unflip")
        else:
            self.flip_button.setText("Flip Horizontal")

app = QApplication(sys.argv)
win = MainWindow()
win.show()
sys.exit(app.exec_())

SystemExit: 0

C:\Users\nehat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sys
import os
import tempfile
import cv2
import numpy as np
import shutil

from PyQt5.QtWidgets import (QApplication, QWidget, QVBoxLayout, QHBoxLayout,
                             QLabel, QPushButton, QComboBox, QFileDialog, QMessageBox)
from PyQt5.QtCore import QTimer, Qt
from PyQt5.QtGui import QImage, QPixmap

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Overlay + Shade Filters - PyQt")
        self.resize(800, 600)

        self.input_mode = "Image"
        self.current_image = None
        self.current_video_path = None
        self.cap = None
        self.timer = QTimer()

        self.processed_video_output_path = None
        self.last_webcam_frame = None
        self.flip_webcam = False

        # Combos for overlay and shade filters
        self.overlay_combo = QComboBox()
        self.overlay_combo.addItems(list(filter_functions.keys()))
        
        self.shade_combo = QComboBox()
        self.shade_combo.addItems(list(shade_filters.keys()))

        self.mode_combo = QComboBox()
        self.mode_combo.addItems(["Image", "Video", "Webcam"])
        self.mode_combo.currentTextChanged.connect(self.on_mode_changed)

        self.load_button = QPushButton("Load")
        self.load_button.clicked.connect(self.on_load_clicked)

        self.apply_button = QPushButton("Apply Filters")
        self.apply_button.clicked.connect(self.on_apply_filters)
        
        self.save_button = QPushButton("Save")
        self.save_button.setEnabled(False)
        self.save_button.clicked.connect(self.on_save_clicked)

        self.take_snapshot_button = QPushButton("Take Snapshot")
        self.take_snapshot_button.setEnabled(False)
        self.take_snapshot_button.clicked.connect(self.on_take_snapshot_clicked)

        self.flip_button = QPushButton("Flip Horizontal")
        self.flip_button.setEnabled(False)
        self.flip_button.clicked.connect(self.on_flip_clicked)

        self.image_label = QLabel("No input loaded.")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setStyleSheet("background-color: #aaa;")

        top_layout = QHBoxLayout()
        top_layout.addWidget(QLabel("Input Type:"))
        top_layout.addWidget(self.mode_combo)
        top_layout.addSpacing(20)

        top_layout.addWidget(QLabel("Overlay Filter:"))
        top_layout.addWidget(self.overlay_combo)
        top_layout.addSpacing(20)

        top_layout.addWidget(QLabel("Shade Filter:"))
        top_layout.addWidget(self.shade_combo)
        top_layout.addSpacing(20)

        top_layout.addWidget(self.load_button)
        top_layout.addWidget(self.apply_button)
        top_layout.addWidget(self.take_snapshot_button)
        top_layout.addWidget(self.flip_button)
        top_layout.addWidget(self.save_button)

        main_layout = QVBoxLayout()
        main_layout.addLayout(top_layout)
        main_layout.addWidget(self.image_label, stretch=1)

        self.setLayout(main_layout)
        self.update_ui_for_mode()

        # Initialize a separate timer for video playback
        self.video_timer = QTimer()
        self.video_timer.timeout.connect(self.update_video_frame)

        # VideoWriter object for saving processed video
        self.video_out = None

    def on_mode_changed(self, text):
        self.input_mode = text
        self.update_ui_for_mode()

    def update_ui_for_mode(self):
        self.save_button.setEnabled(False)
        self.take_snapshot_button.setEnabled(False)
        self.flip_button.setEnabled(False)
        if self.input_mode == "Image":
            self.load_button.setText("Load Image")
            self.stop_webcam()
            self.image_label.setText("No image loaded.")
            self.current_image = None
            self.current_video_path = None
        elif self.input_mode == "Video":
            self.load_button.setText("Load Video")
            self.stop_webcam()
            self.image_label.setText("No video loaded.")
            self.current_image = None
            self.current_video_path = None
        elif self.input_mode == "Webcam":
            self.load_button.setText("Start Webcam")
            self.current_image = None
            self.current_video_path = None
            self.start_webcam()
            self.take_snapshot_button.setEnabled(True)
            self.flip_button.setEnabled(True)

    def on_load_clicked(self):
        self.save_button.setEnabled(False)
        if self.input_mode == "Image":
            path, _ = QFileDialog.getOpenFileName(self, "Select Image", "", "Images (*.png *.jpg *.jpeg *.bmp)")
            if path:
                img = cv2.imread(path)
                if img is not None:
                    self.current_image = img
                    self.show_image(img)
                else:
                    QMessageBox.warning(self, "Error", "Failed to load image.")
        elif self.input_mode == "Video":
            path, _ = QFileDialog.getOpenFileName(self, "Select Video", "", "Video Files (*.mp4 *.avi *.mov)")
            if path:
                self.current_video_path = path
                cap = cv2.VideoCapture(path)
                ret, frame = cap.read()
                cap.release()
                if ret:
                    self.show_image(frame)
                else:
                    QMessageBox.warning(self, "Error", "Failed to load video.")
        elif self.input_mode == "Webcam":
            # Toggle webcam on/off
            if self.cap is None:
                self.start_webcam()
            else:
                self.stop_webcam()

    def start_webcam(self):
        if self.cap is not None:
            return
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            QMessageBox.warning(self, "Error", "Failed to access webcam.")
            self.cap = None
            return
        self.timer.timeout.connect(self.update_webcam_frame)
        self.timer.start(30)
        self.load_button.setText("Stop Webcam")
        self.take_snapshot_button.setEnabled(True)
        self.flip_button.setEnabled(True)

    def stop_webcam(self):
        if self.cap is not None:
            self.timer.stop()
            self.cap.release()
            self.cap = None
            self.load_button.setText("Start Webcam")
            self.image_label.setText("Webcam stopped.")
            self.take_snapshot_button.setEnabled(False)
            self.flip_button.setEnabled(False)

    def update_webcam_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return
        if self.flip_webcam:
            frame = cv2.flip(frame, 1)
        frame = self.apply_filters_to_frame(frame)
        self.show_image(frame)
        self.last_webcam_frame = frame

    def apply_filters_to_frame(self, frame):
        # Apply overlay filter first
        overlay_name = self.overlay_combo.currentText()
        overlay_func = filter_functions.get(overlay_name, lambda x: x)
        frame = overlay_func(frame)

        # Then apply shade filter
        shade_name = self.shade_combo.currentText()
        shade_func = shade_filters.get(shade_name, lambda x: x)
        frame = shade_func(frame)

        return frame

    def show_image(self, image):
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb.shape
        bytes_per_line = ch * w
        qimg = QImage(rgb.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pix = QPixmap.fromImage(qimg)
        self.image_label.setPixmap(pix.scaled(self.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))

    def on_apply_filters(self):
        if self.input_mode == "Image":
            if self.current_image is None:
                QMessageBox.warning(self, "No Image", "Load an image first.")
                return
            processed = self.apply_filters_to_frame(self.current_image)
            self.show_image(processed)
            self.current_image = processed
            self.save_button.setEnabled(True)

        elif self.input_mode == "Video":
            if self.current_video_path is None:
                QMessageBox.warning(self, "No Video", "Load a video first.")
                return

            # Open the video file
            self.cap_video = cv2.VideoCapture(self.current_video_path)
            if not self.cap_video.isOpened():
                QMessageBox.warning(self, "Error", "Failed to open video.")
                return

            # Get video properties
            frame_width = int(self.cap_video.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(self.cap_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = self.cap_video.get(cv2.CAP_PROP_FPS)
            if fps == 0:
                fps = 30  # Fallback FPS

            # Define the codec and create VideoWriter object
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            temp_dir = tempfile.gettempdir()
            output_path = os.path.join(temp_dir, 'processed_video_output.mp4')
            self.video_out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

            # Start the video playback timer
            self.video_timer.start(int(1000 / fps))

            QMessageBox.information(self, "Info", "Started video playback with filters.")
            self.save_button.setEnabled(True)

        elif self.input_mode == "Webcam":
            QMessageBox.information(self, "Info", "Filter applied in real-time to webcam feed.\nUse 'Take Snapshot' to capture a frame.")

    def process_video_file(self, video_path):
        overlay_name = self.overlay_combo.currentText()
        overlay_func = filter_functions.get(overlay_name, lambda x: x)

        shade_name = self.shade_combo.currentText()
        shade_func = shade_filters.get(shade_name, lambda x: x)

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return None

        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        temp_dir = tempfile.gettempdir()
        output_path = os.path.join(temp_dir, 'processed_video_output.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = overlay_func(frame)
            frame = shade_func(frame)
            if len(frame.shape) == 2:
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
            out.write(frame)

        cap.release()
        out.release()
        return output_path

    def on_save_clicked(self):
        if self.input_mode == "Image" and self.current_image is not None:
            path, _ = QFileDialog.getSaveFileName(self, "Save Image", "", "Images (*.png *.jpg *.jpeg *.bmp)")
            if path:
                cv2.imwrite(path, self.current_image)
                QMessageBox.information(self, "Saved", f"Image saved at:\n{path}")

        elif self.input_mode == "Video":
            if self.processed_video_output_path is None:
                QMessageBox.warning(self, "No Processed Video", "There is no processed video to save.")
                return

            path, _ = QFileDialog.getSaveFileName(self, "Save Video", "", "Video Files (*.mp4 *.avi)")
            if path:
                try:
                    shutil.copyfile(self.processed_video_output_path, path)
                    QMessageBox.information(self, "Saved", f"Video saved at:\n{path}")
                except Exception as e:
                    QMessageBox.warning(self, "Error", f"Failed to save video.\n{e}")

        elif self.input_mode == "Webcam" and self.current_image is not None:
            path, _ = QFileDialog.getSaveFileName(self, "Save Snapshot", "", "Images (*.png *.jpg *.jpeg *.bmp)")
            if path:
                cv2.imwrite(path, self.current_image)
                QMessageBox.information(self, "Saved", f"Snapshot saved at:\n{path}")
        else:
            QMessageBox.information(self, "Info", "Nothing to save at this moment.")

    def on_take_snapshot_clicked(self):
        # If we have last_webcam_frame stored, treat it as current_image.
        if self.input_mode == "Webcam" and self.last_webcam_frame is not None:
            self.current_image = self.last_webcam_frame.copy()
            self.save_button.setEnabled(True)
            QMessageBox.information(self, "Snapshot", "Snapshot taken and ready to save!")
        else:
            QMessageBox.information(self, "Info", "No webcam frame available to snapshot.")

    def on_flip_clicked(self):
        self.flip_webcam = not self.flip_webcam
        if self.flip_webcam:
            self.flip_button.setText("Unflip")
        else:
            self.flip_button.setText("Flip Horizontal")

    def update_video_frame(self):
        ret, frame = self.cap_video.read()
        if not ret:
            # End of video
            self.video_timer.stop()
            self.cap_video.release()
            if self.video_out is not None:
                self.video_out.release()
                self.video_out = None
                self.processed_video_output_path = os.path.join(tempfile.gettempdir(), 'processed_video_output.mp4')
                QMessageBox.information(self, "Info", f"Finished video playback.\nProcessed video saved at:\n{self.processed_video_output_path}")
            return

        if self.flip_webcam:
            frame = cv2.flip(frame, 1)

        # Apply selected filters to the frame
        processed_frame = self.apply_filters_to_frame(frame)

        # Display the processed frame in the UI
        self.show_image(processed_frame)

        # Write the processed frame to the output video
        if self.video_out is not None:
            self.video_out.write(processed_frame)

app = QApplication(sys.argv)
win = MainWindow()
win.show()
sys.exit(app.exec_())

2024-12-07 10:52:11.227 Python[53732:20969702] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
2024-12-07 10:52:13.527 Python[53732:20969702] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
/var/folders/gg/w5v2fbr54774543fcpcync1c0000gn/T/ipykernel_53732/2712805928.py:224: RuntimeWarning: overflow encountered in scalar subtract
  quant_error = oldpixel - newpixel
